In [1]:
#importing stuff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols


In [2]:
#importing data
deliveries= pd.read_csv('deliveries.csv')
matches= pd.read_csv('matches.csv')


In [3]:
#adding stuff to 'deliveries'
deliveries['is_legal']= 1-((deliveries.noball_runs + deliveries.wide_runs)>.5)
deliveries['is_dismissed']= 1- (deliveries.player_dismissed.isnull())
deliveries['is_dot']=1*(deliveries.total_runs<.1)


In [4]:
#determining match result
matches['win']=1*(matches.win_by_runs>0)

In [5]:
#adding over by over data to dict
overs_dict={}
for i in deliveries.match_id.unique():
    df_match= deliveries[deliveries.match_id==i]
    
    df_1= df_match[df_match.inning==1]
    
    df_2= df_match[df_match.inning==2]
    
    overs_dict[i]={}
    
    for ov in df_1.over.unique():
        overs_dict[i][str(100+ov)+ '_runs']=sum(df_1[df_1.over==ov].total_runs)
        overs_dict[i][str(100+ov)+ '_wickets']=sum(df_1[df_1.over==ov].is_dismissed)
        overs_dict[i][str(100+ov)+ '_dots']= sum(df_1[df_1.over==ov].is_dot)
        
    for ov in df_2.over.unique():
        overs_dict[i][str(200+ov)+'_runs']=sum(df_2[df_2.over==ov].total_runs)
        overs_dict[i][str(200+ov)+'_wickets']=sum(df_2[df_2.over==ov].is_dismissed)
        overs_dict[i][str(200+ov)+'_dots']=sum(df_2[df_2.over==ov].is_dot)

    overs_dict[i]['id']=i

    

In [6]:
#turning the over dict into a df

over_df= pd.DataFrame(overs_dict).T
over_df.head()

,101_dots,101_runs,101_wickets,102_dots,102_runs,102_wickets,103_dots,103_runs,103_wickets,104_dots,...,218_dots,218_runs,218_wickets,219_dots,219_runs,219_wickets,220_dots,220_runs,220_wickets,id
1,4.0,7.0,0.0,2.0,16.0,1.0,2.0,6.0,0.0,2.0,...,5.0,2.0,2.0,2.0,9.0,1.0,2.0,7.0,1.0,1.0
2,3.0,11.0,0.0,4.0,8.0,0.0,3.0,9.0,0.0,3.0,...,2.0,7.0,0.0,2.0,7.0,0.0,0.0,15.0,0.0,2.0
3,4.0,8.0,0.0,2.0,8.0,0.0,3.0,6.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
4,5.0,1.0,1.0,4.0,2.0,0.0,2.0,4.0,0.0,1.0,...,1.0,12.0,0.0,3.0,7.0,0.0,NaN,NaN,NaN,4.0
5,2.0,10.0,0.0,3.0,9.0,0.0,3.0,7.0,0.0,5.0,...,1.0,11.0,0.0,4.0,2.0,0.0,3.0,3.0,2.0,5.0


In [7]:
#making the aggregate scores dict for the innings

scores_dict={}
for i in deliveries.match_id.unique():
    
    df_match= deliveries[deliveries.match_id==i]
    
    df_1= df_match[df_match.inning==1]
    
    df_2= df_match[df_match.inning==2]
    
    df_last= df_match.tail(1)
    
    ov= np.array(df_last.over)[0]
    
    ov_df= df_2[df_2.over==ov]
    ba=sum(ov_df.is_legal)
    
    ov=20-ov
    
    ba=max(6-ba, 0)
    
    scores_dict[i]={}
    
    scores_dict[i]['id']= i
    
    scores_dict[i]['innings1']= df_1.total_runs.sum()
    
    scores_dict[i]['wickets1']= df_1.is_dismissed.sum()
    
    scores_dict[i]['maxover1']= max(df_1.over)
    
    scores_dict[i]['innings2']= df_2.total_runs.sum()
    
    scores_dict[i]['rem']= (scores_dict[i]['innings2']>scores_dict[i]['innings1'])*(6*ov + ba)
    
    scores_dict[i]['mar']= max(0,scores_dict[i]['innings2']- scores_dict[i]['innings1'])
    +  np.round(100*(scores_dict[i]['rem']* scores_dict[i]['innings1'])/120)/100

In [8]:
#Making a df out of the scores data
scores_df= pd.DataFrame(scores_dict).T

In [9]:
#merging matches and scores
match_df= pd.merge(matches, scores_df, on='id', how='right')

In [10]:
#merging match and over
match_df= pd.merge(match_df, over_df, on='id', how='right')

In [11]:
#splitting innings into quarters and getting runs
match_df['qruns11']= 0
match_df['qruns12']= 0
match_df['qruns13']= 0
match_df['qruns14']= 0
count=0
num=0
for i in range(1,21):
    if count%5==0:
        num+=1
    match_df['qruns'+'1'+str(num)]= match_df['qruns'+'1'+str(num)]+ match_df[str(100+i)+'_runs']
    count+=1


In [12]:
#getting wickets for quarters
match_df['qwickets11']= 0
match_df['qwickets12']= 0
match_df['qwickets13']= 0
match_df['qwickets14']= 0
count=0
num=0
for i in range(1,21):
    if count%5==0:
        num+=1
    match_df['qwickets'+'1'+str(num)]= match_df['qwickets'+'1'+str(num)]+ match_df[str(100+i)+'_wickets']
    count+=1


In [13]:
#getting dots for quarters
match_df['qdots11']= 0
match_df['qdots12']= 0
match_df['qdots13']= 0
match_df['qdots14']= 0
count=0
num=0
for i in range(1,21):
    if count%5==0:
        num+=1
    match_df['qdots'+'1'+str(num)]= match_df['qdots'+'1'+str(num)]+ match_df[str(100+i)+'_dots']
    count+=1


In [15]:
#filtering out matches with rain, no result, or tied
no_rain= match_df[match_df.dl_applied==0]
no_rain=no_rain[no_rain.result=='normal']
no_rain['islegit']= (no_rain.wickets1>9.5) + (no_rain.maxover1>19.5)
no_rain=no_rain[no_rain['islegit']]

/anaconda3/lib/python3.6/site-packages/pandas/core/computation/expressions.py:180: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [16]:
#Adding the chasing team's target
no_rain['target']=no_rain.innings1+1



In [17]:
#Adding data for runs/wickets/dots until a certain over
for i in range(2,22):
    no_rain['wickets_until_'+str(100+i-1)]=0
    no_rain['wickets_until_'+str(200+i-1)]=0
    no_rain['runs_until_'+str(100+i-1)]=0
    no_rain['runs_until_'+str(200+i-1)]=0
    no_rain['dots_until_'+str(100+i-1)]=0
    no_rain['dots_until_'+str(200+i-1)]=0
    for j in range(1,i):
        no_rain['wickets_until_'+str(100+i-1)]+=no_rain[str(100+j)+'_wickets']
        no_rain['wickets_until_'+str(200+i-1)]+=no_rain[str(200+j)+'_wickets']
        no_rain['runs_until_'+str(100+i-1)]+=no_rain[str(100+j)+'_runs']
        no_rain['runs_until_'+str(200+i-1)]+=no_rain[str(200+j)+'_runs']
        no_rain['dots_until_'+str(100+i-1)]+=no_rain[str(100+j)+'_dots']
        no_rain['dots_until_'+str(200+i-1)]+=no_rain[str(200+j)+'_dots']
        

In [18]:
#Number of runs left to score, for 2nd innings
for i in range(1,20):
    no_rain['runs_left_'+str(200+i)]= no_rain.target - no_rain['runs_until_'+str(200+i)]
    

In [19]:
#defining wickets and dots columns for lin reg.

wickets_cols='qwickets11+'+ 'qwickets12+' + 'qwickets13+' + 'qwickets14'
dots_cols='qdots11+'+'qdots12+'+'qdots13+'+'qdots14'

In [30]:
#Linear model for wickets
lr_model_wickets= ols(formula= 'innings1~'+wickets_cols, data= no_rain[no_rain.season>2013.5]).fit()
lr_model_wickets.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               innings1   R-squared:                       0.472
Model:                            OLS   Adj. R-squared:                  0.462
Method:                 Least Squares   F-statistic:                     46.93
Date:                Thu, 18 Apr 2019   Prob (F-statistic):           3.82e-28
Time:                        20:18:15   Log-Likelihood:                -932.59
No. Observations:                 215   AIC:                             1875.
Df Residuals:                     210   BIC:                             1892.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    215.4005      4.078     52.822      0.000     207.362     223.439
qwickets11   -15.9209      1.463    -10.881      0.000     -18.805     -13.037
qwickets12   -11.9465      1.361     -8.775      0.000     -14.630      -9.263
qwickets13    -7.9640      1.385     -5.748      0.000     -10.695      -5.233
qwickets14    -3.2391      0.898     -3.606      0.000      -5.010      -1.468
==============================================================================
Omnibus:                        3.216   Durbin-Watson:                   2.072
Prob(Omnibus):                  0.200   Jarque-Bera (JB):                2.837
Skew:                           0.262   Prob(JB):                        0.242
Kurtosis:                       3.207   Cond. No.                         12.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
#Testing for anomaly in final quarter wickets lost, no anomaly

lr_model_test= ols(formula= 'qwickets14~wickets_until_115', data= no_rain[no_rain.season>2013.5]).fit()
lr_model_test.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             qwickets14   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                  0.001497
Date:                Thu, 18 Apr 2019   Prob (F-statistic):              0.969
Time:                        20:18:48   Log-Likelihood:                -383.74
No. Observations:                 215   AIC:                             771.5
Df Residuals:                     213   BIC:                             778.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             2.6231      0.257     10.220      0.000       2.117       3.129
wickets_until_115    -0.0028      0.072     -0.039      0.969      -0.144       0.138
==============================================================================
Omnibus:                        2.471   Durbin-Watson:                   2.081
Prob(Omnibus):                  0.291   Jarque-Bera (JB):                2.066
Skew:                           0.116   Prob(JB):                        0.356
Kurtosis:                       2.580   Cond. No.                         9.95
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
#Running Regression with dots and wickets for each quarter

lr_model_dot_wickets= ols(formula= 'innings1~'+wickets_cols+'+'+dots_cols, data= no_rain[no_rain.season>2013.5]).fit()
lr_model_dot_wickets.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               innings1   R-squared:                       0.550
Model:                            OLS   Adj. R-squared:                  0.533
Method:                 Least Squares   F-statistic:                     31.53
Date:                Thu, 18 Apr 2019   Prob (F-statistic):           5.71e-32
Time:                        20:19:22   Log-Likelihood:                -915.30
No. Observations:                 215   AIC:                             1849.
Df Residuals:                     206   BIC:                             1879.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    245.6962      7.029     34.956      0.000     231.839     259.554
qwickets11   -11.2544      1.622     -6.941      0.000     -14.451      -8.058
qwickets12    -7.9889      1.483     -5.389      0.000     -10.912      -5.066
qwickets13    -4.8697      1.475     -3.300      0.001      -7.779      -1.961
qwickets14    -1.6429      0.990     -1.659      0.099      -3.596       0.310
qdots11       -0.8064      0.429     -1.881      0.061      -1.652       0.039
qdots12       -0.9316      0.386     -2.416      0.017      -1.692      -0.172
qdots13       -1.3380      0.413     -3.237      0.001      -2.153      -0.523
qdots14       -1.8326      0.576     -3.183      0.002      -2.968      -0.697
==============================================================================
Omnibus:                        4.954   Durbin-Watson:                   2.065
Prob(Omnibus):                  0.084   Jarque-Bera (JB):                4.775
Skew:                           0.364   Prob(JB):                       0.0919
Kurtosis:                       3.063   Cond. No.                         130.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
#Seeing the number of dots added by each wicket

lr_model_all_dots= ols(formula= 'dots_until_120~'+wickets_cols, data= no_rain[no_rain.season>2013.5]).fit()
lr_model_all_dots.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         dots_until_120   R-squared:                       0.364
Model:                            OLS   Adj. R-squared:                  0.352
Method:                 Least Squares   F-statistic:                     30.02
Date:                Thu, 18 Apr 2019   Prob (F-statistic):           9.32e-20
Time:                        20:20:03   Log-Likelihood:                -699.63
No. Observations:                 215   AIC:                             1409.
Df Residuals:                     210   BIC:                             1426.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     28.6121      1.380     20.734      0.000      25.892      31.332
qwickets11     4.3546      0.495      8.795      0.000       3.379       5.331
qwickets12     3.2085      0.461      6.964      0.000       2.300       4.117
qwickets13     2.1838      0.469      4.658      0.000       1.260       3.108
qwickets14     0.8035      0.304      2.643      0.009       0.204       1.403
==============================================================================
Omnibus:                        2.362   Durbin-Watson:                   1.660
Prob(Omnibus):                  0.307   Jarque-Bera (JB):                2.433
Skew:                           0.234   Prob(JB):                        0.296
Kurtosis:                       2.772   Cond. No.                         12.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""